# Example tree for Figure 1

In [57]:
import toytree as tt
import numpy as np
import pandas as pd
import toyplot as tp
import toyplot.svg

In [68]:
newick_example = "(((T1, T2)N4, (T3, T4, T5, T6)N5)N2,((T7, T8)N6, (T9, T10)N7, (T11, T12, T13)N8)N3)N1;"
ex_tree = tt.tree(newick=newick_example, tree_format=8)

In [69]:
canvas = tp.Canvas(width=800, height=400)
ax0 = canvas.cartesian(bounds=(0, 800, 0, 400), padding=0)

ex_tree.draw(
    layout="d",
    edge_type="c",
    node_sizes=[30 for i in ex_tree.get_node_values("name", 1, 1)],
    node_style={"stroke": "black"},
    node_labels=[x for x in ex_tree.get_node_values("name", 1, 1)],
    node_labels_style={"font-size": 12},
    tip_labels=False,
    node_colors=["lightblue" if i in ["N5", "T9", "T4"] else "lightgrey" for i in ex_tree.get_node_values("name", 1, 1)],
    width=800,
    height=400,
    axes=ax0
)
tp.svg.render(canvas, "./fig1_tree.svg")

In [70]:
for n in ex_tree.treenode.traverse():
    print(n.name)

N1
N2
N3
N4
N5
N6
N7
N8
T1
T2
T3
T4
T5
T6
T7
T8
T9
T10
T11
T12
T13
